In [75]:
import json
import PIL
import copy
import os

In [76]:
target_dir = "./second_test"
target_name = "test_07.json"

line_strip_path = os.path.join(target_dir, target_name)
with open(line_strip_path) as f:
    line_strip = json.load(f)

In [77]:
# 画像手前をセグメンテーションに入れない場合
# 出力用辞書
copy_dict = copy.deepcopy(line_strip)

left_dict = copy_dict['shapes'][0]
right_dict = copy_dict['shapes'][1]
right_dict['points'].reverse()  # 右ラインを逆順にしないとpolygonにならない

left_dict["points"].extend(right_dict["points"])  # ラインの座標を連結

# 左ラインだった場所をpolygon情報に上書き
copy_dict['shapes'][0]['label'] = "road"
copy_dict['shapes'][0]['points'] = left_dict['points']
copy_dict['shapes'][0]['shape_type'] = 'polygon'

# 右ライン情報を削除
del copy_dict['shapes'][1]

In [78]:
X_MAX = 640
Y_MAX = 480
# 画像手前をセグメンテーションに入れる場合
# 座標は, 画像左上を原点とし, 右方向がx正, 下方向がy正
# 出力用辞書
copy_dict = copy.deepcopy(line_strip)

left_dict = copy_dict['shapes'][0]
right_dict = copy_dict['shapes'][1]

left_first_point = left_dict["points"][-1]  # 一番手前の点
left_second_point = left_dict["points"][-2]  # 二番目に手前の点

# 大抵の場合負の値になる, yの増加量/xの増加量
left_slope = (left_first_point[1] - left_second_point[1]) \
    / (left_first_point[0] - left_second_point[0])

# 直線の延長が先に画像の左端に当たった場合
if (0 - left_first_point[0])*left_slope + left_first_point[1] < Y_MAX:
    left_added_point = [[0, (0 - left_first_point[0])*left_slope + left_first_point[1]],
     [0, Y_MAX]]
# 直線の延長が先に画像の下端に当たった場合
else:
    left_added_point = [[left_first_point[0] + (Y_MAX - left_first_point[1])/left_slope ,Y_MAX]]


right_first_point = right_dict["points"][-1]  # 一番手前の点
right_second_point = right_dict["points"][-2]  # 二番目に手前の点

# 大抵の場合正の値になる, yの増加量/xの増加量
right_slope = (right_first_point[1] - right_second_point[1]) \
    / (right_first_point[0] - right_second_point[0])

# 直線の延長が先に画像の右端に当たった場合
if (X_MAX - right_first_point[0])*right_slope + right_first_point[1] < Y_MAX:
    right_added_point = [[X_MAX, (X_MAX - right_first_point[0])*right_slope + right_first_point[1]],
    [X_MAX, Y_MAX]]
# 直線の延長が先に画像の下端に当たった場合
else:
    right_added_point = [[right_first_point[0] + (Y_MAX - right_first_point[1])/right_slope ,Y_MAX]]

print(left_added_point)
print(right_added_point)

left_dict['points'].extend(left_added_point)
right_dict['points'].extend(right_added_point)
right_dict['points'].reverse()

left_dict["points"].extend(right_dict["points"])  # ラインの座標を連結

# 左ラインだった場所をpolygon情報に上書き
copy_dict['shapes'][0]['label'] = "road"
copy_dict['shapes'][0]['points'] = left_dict['points']
copy_dict['shapes'][0]['shape_type'] = 'polygon'

# 右ライン情報を削除
del copy_dict['shapes'][1]

[[0, 440.10289992927], [0, 480]]
[[640, 438.5983320831312], [640, 480]]


In [79]:
# jsonファイルの書き出し
convert_name = "converted_" + target_name
output_path = os.path.join(target_dir, convert_name)
with open(output_path, "w") as f:
    json.dump(copy_dict, f)

In [81]:
import glob
glob.glob("./second_test/*json")

['./second_test/test_01.json',
 './second_test/test_16.json',
 './second_test/test_11.json',
 './second_test/test_07.json',
 './second_test/test_06.json',
 './second_test/test_10.json',
 './second_test/test_05.json',
 './second_test/test_13.json',
 './second_test/test_09.json',
 './second_test/test_08.json',
 './second_test/test_12.json',
 './second_test/test_04.json',
 './second_test/test_03.json',
 './second_test/test_15.json',
 './second_test/test_14.json',
 './second_test/test_02.json']

In [83]:
"abc_sss".find("afc")

-1